In [ ]:
pip install beautifulsoup4


In [ ]:
pip install selenium

we need requests to be able to send request to the website and get a response back

In [ ]:
pip install requests

In [ ]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


Now lets scrape our data


In [ ]:
from bs4 import BeautifulSoup
import requests
import re
import json
import pandas as pd

Lets create a for loop to look through all the pages and create a data frame

In [ ]:
Allpages = []
#lets create a for list to grab page info and append to our list
for i in range(41):
    url = 'https://www.rightmove.co.uk/house-prices/sunderland.html?soldIn=1&page={0}'.format(i)
    page = requests.get(url)
    #response statsu code is 200, this means its succesful
    soup = BeautifulSoup(page.content, 'html.parser')
    #content of the page and the parser for html
    script_text = soup.find(lambda tag: tag.name=="script" and '__PRELOADED_STATE__' in tag.text)
    #Now we tell python to grap the  texts that are in the script tags
    json_object = re.search('window\.__PRELOADED_STATE__ \= (\{.+\})', str(script_text)).group(1)
    #we tell python to find the regular expressions above and remove them
    info = json.loads(json_object)
    #to parse json object and return the python dictionary object
    my_dict = pd.json_normalize(data=info['results']['properties'])
    #now we turn this dictionary into a table
    Allpages.append(my_dict)
    # here we append each table to the list. this gives us a list of tables
    


on below code we can see that the data is stored in a dictionary under the script tag.

In [ ]:
soup

In [ ]:
script_text

In [ ]:
json_object

In [ ]:
info

In [ ]:
Allpages

Now lets concatinate the data frame into one

In [ ]:
My_dataframe = pd.concat(Allpages)
My_dataframe.reset_index(inplace=True)
My_dataframe.head(2)

,index,address,propertyType,bedrooms,hasFloorPlan,transactions,detailUrl,images.imageUrl,images.count,location.lat,location.lng
0,0,"West Hendon House, Ryhope Road, Sunderland, Ty...",Detached,NaN,False,"[{'displayPrice': '£139,240', 'dateSold': '20 ...",,/spw/images/placeholder/no-image.svg,0,54.89689,-1.38095
1,1,"7, Avonmouth Road, Sunderland, Tyne And Wear S...",Semi-Detached,3.0,True,"[{'displayPrice': '£92,500', 'dateSold': '16 J...",https://www.rightmove.co.uk/house-prices/detai...,https://media.rightmove.co.uk/dir/30k/29429/86...,14,54.87467,-1.43147


Lets drop some the images and unhelpful indexes of the tables

In [ ]:
My_dataframe = My_dataframe.drop(['index'], axis = 1)
My_dataframe = My_dataframe.drop(['images.imageUrl'], axis = 1)
My_dataframe.head

KeyError: ignored

Lets create a dataframe with languistic features


In [ ]:
column_names = ["id", "details"]
#lets create thel labels for the columns
Myfeatures = pd.DataFrame(columns = column_names)


for i in range(1025):
  #we use 1000 because out table above has 1000 rows. and we are grabbing url from it
    newUrl = My_dataframe.loc[i,'detailUrl']
    if not newUrl:
        other_features = pd.DataFrame({'id': [i],
                                'details': ['Na']})
    else:
        detail_page = requests.get(newUrl)
        detail_soup = BeautifulSoup(detail_page.content, 'html.parser')
        lists = detail_soup.find_all('body', class_='propertyDetails solddetails compact odin-top-nav')
        for alist in lists:
            features = alist.find('ul', class_="keyfeatures")
        other_features = pd.DataFrame({'id': [i],
                                'details': [features]})
    Myfeatures = Myfeatures.append(other_features)

Myfeatures


,id,details
0,0,Na
0,1,"[\n, [Semi Detached House], \n, [3 Bedrooms], ..."
0,2,Na
0,3,"[\n, [2 BED SEMI ], \n, [VERY POPULAR FARRINGD..."
0,4,None
...,...,...
0,1020,"[\n, [Detached House], \n, [3 Bedrooms], \n, [..."
0,1021,"[\n, [360 VIRTUAL TOUR], \n, [THREE BEDROOM SE..."
0,1022,Na
0,1023,"[\n, [Superb Mid Terraced House], \n, [Compreh..."


Lets create our  dataframe withall the feaatures

lets join

In [ ]:
#lets set the column id to be the index
gaga = Myfeatures.set_index('id')

final_data = pd.merge(My_dataframe, gaga, left_index=True, right_index=True)
final_data

,address,propertyType,bedrooms,hasFloorPlan,transactions,detailUrl,images.count,location.lat,location.lng,details
0,"West Hendon House, Ryhope Road, Sunderland, Ty...",Detached,NaN,False,"[{'displayPrice': '£139,240', 'dateSold': '20 ...",,0,54.89689,-1.38095,Na
1,"7, Avonmouth Road, Sunderland, Tyne And Wear S...",Semi-Detached,3.0,True,"[{'displayPrice': '£92,500', 'dateSold': '16 J...",https://www.rightmove.co.uk/house-prices/detai...,14,54.87467,-1.43147,"[\n, [Semi Detached House], \n, [3 Bedrooms], ..."
2,"59, Flint Road, Sunderland, Tyne And Wear SR4 6EG",Semi-Detached,NaN,False,"[{'displayPrice': '£110,000', 'dateSold': '16 ...",,0,54.90903,-1.40182,Na
3,"2, Aldwych Road, Sunderland, Tyne And Wear SR3...",Semi-Detached,2.0,False,"[{'displayPrice': '£75,000', 'dateSold': '15 J...",https://www.rightmove.co.uk/house-prices/detai...,11,54.87053,-1.43169,"[\n, [2 BED SEMI ], \n, [VERY POPULAR FARRINGD..."
4,"71, Bede Street, Sunderland, Tyne And Wear SR6...",Terraced,2.0,False,"[{'displayPrice': '£94,000', 'dateSold': '14 J...",https://www.rightmove.co.uk/house-prices/detai...,11,54.92155,-1.37090,None
...,...,...,...,...,...,...,...,...,...,...
995,"31, Dykelands Road, Sunderland, Tyne And Wear ...",Semi-Detached,3.0,True,"[{'displayPrice': '£240,000', 'dateSold': '12 ...",https://www.rightmove.co.uk/house-prices/detai...,20,54.93283,-1.38024,"[\n, [Semi-Detached Home], \n, [Three Good Siz..."
996,"56, Queen Alexandra Road, Sunderland, Tyne And...",Semi-Detached,2.0,True,"[{'displayPrice': '£153,000', 'dateSold': '12 ...",https://www.rightmove.co.uk/house-prices/detai...,12,54.88697,-1.37565,"[\n, [Semi Detached Bungalow in Popular and Es..."
997,"11, Redruth Square, Sunderland, Tyne And Wear ...",Semi-Detached,2.0,False,"[{'displayPrice': '£69,950', 'dateSold': '11 N...",https://www.rightmove.co.uk/house-prices/detai...,11,54.92436,-1.43140,"[\n, [Quiet Cul De Sac Location], \n, [No Chai..."
998,"15, Priory Grove, Sunderland, Tyne And Wear SR...",Semi-Detached,3.0,False,"[{'displayPrice': '£210,000', 'dateSold': '11 ...",https://www.rightmove.co.uk/house-prices/detai...,21,54.90020,-1.41396,"[\n, [3 BEDROOMS], \n, [SEMI DETACHED BUNGALOW..."


lets split the transactions because this column is a list of the different transactions the house has had over the years

In [ ]:
trans = final_data['transactions']
for i, val in enumerate(final_data['transactions']):
    #trans[i] = val
    #print(i,val[0])
    trans[i] = val[0] 

transact = pd.DataFrame(trans.values.tolist(), index=trans.index)
transact

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,displayPrice,dateSold,tenure,newBuild
0,"£139,240",20 Jul 2021,Leasehold,False
1,"£92,500",16 Jul 2021,Freehold,False
2,"£110,000",16 Jul 2021,Freehold,False
3,"£75,000",15 Jul 2021,Freehold,False
4,"£94,000",14 Jul 2021,Leasehold,False
...,...,...,...,...
995,"£240,000",12 Nov 2020,Leasehold,False
996,"£153,000",12 Nov 2020,Leasehold,False
997,"£69,950",11 Nov 2020,Freehold,False
998,"£210,000",11 Nov 2020,Freehold,False


lets drop the transcations and also concatinate with the main dataframe

In [ ]:

final_data = pd.merge(final_data, transact, left_index=True, right_index=True)
final_data = final_data.drop(['transactions'], axis=1)
final_data

,address,propertyType,bedrooms,hasFloorPlan,detailUrl,images.count,location.lat,location.lng,details,displayPrice,dateSold,tenure,newBuild
0,"West Hendon House, Ryhope Road, Sunderland, Ty...",Detached,NaN,False,,0,54.89689,-1.38095,Na,"£139,240",20 Jul 2021,Leasehold,False
1,"7, Avonmouth Road, Sunderland, Tyne And Wear S...",Semi-Detached,3.0,True,https://www.rightmove.co.uk/house-prices/detai...,14,54.87467,-1.43147,"[\n, [Semi Detached House], \n, [3 Bedrooms], ...","£92,500",16 Jul 2021,Freehold,False
2,"59, Flint Road, Sunderland, Tyne And Wear SR4 6EG",Semi-Detached,NaN,False,,0,54.90903,-1.40182,Na,"£110,000",16 Jul 2021,Freehold,False
3,"2, Aldwych Road, Sunderland, Tyne And Wear SR3...",Semi-Detached,2.0,False,https://www.rightmove.co.uk/house-prices/detai...,11,54.87053,-1.43169,"[\n, [2 BED SEMI ], \n, [VERY POPULAR FARRINGD...","£75,000",15 Jul 2021,Freehold,False
4,"71, Bede Street, Sunderland, Tyne And Wear SR6...",Terraced,2.0,False,https://www.rightmove.co.uk/house-prices/detai...,11,54.92155,-1.37090,None,"£94,000",14 Jul 2021,Leasehold,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,"31, Dykelands Road, Sunderland, Tyne And Wear ...",Semi-Detached,3.0,True,https://www.rightmove.co.uk/house-prices/detai...,20,54.93283,-1.38024,"[\n, [Semi-Detached Home], \n, [Three Good Siz...","£240,000",12 Nov 2020,Leasehold,False
996,"56, Queen Alexandra Road, Sunderland, Tyne And...",Semi-Detached,2.0,True,https://www.rightmove.co.uk/house-prices/detai...,12,54.88697,-1.37565,"[\n, [Semi Detached Bungalow in Popular and Es...","£153,000",12 Nov 2020,Leasehold,False
997,"11, Redruth Square, Sunderland, Tyne And Wear ...",Semi-Detached,2.0,False,https://www.rightmove.co.uk/house-prices/detai...,11,54.92436,-1.43140,"[\n, [Quiet Cul De Sac Location], \n, [No Chai...","£69,950",11 Nov 2020,Freehold,False
998,"15, Priory Grove, Sunderland, Tyne And Wear SR...",Semi-Detached,3.0,False,https://www.rightmove.co.uk/house-prices/detai...,21,54.90020,-1.41396,"[\n, [3 BEDROOMS], \n, [SEMI DETACHED BUNGALOW...","£210,000",11 Nov 2020,Freehold,False


Lets save our data into a csv file

In [ ]:
from google.colab import files
final_data.to_csv('FinalMove.csv') 
files.download('FinalMove.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>